In [92]:
import os
import numpy as np
import pandas as pd
import json
import ast
from pandas.io.json import json_normalize

path = '/content/query_result_2020-07-10T11_58_44.440082Z.csv'

In [93]:
df = pd.read_csv(path)

In [94]:
df = pd.concat([df['User ID'],df['Response']],axis=1)
df.head(2)

,User ID,Response
0,394897,"{u'status': u'Rejected', u'uploadTime': None, ..."
1,104832,"{u'status': u'Submitted', u'uploadTime': 15942..."


In [95]:
df['Response'] = df['Response'].apply(lambda x : ast.literal_eval(str(x)))

In [96]:
df.head(2)

,User ID,Response
0,394897,"{'status': 'Rejected', 'uploadTime': None, 'do..."
1,104832,"{'status': 'Submitted', 'uploadTime': 15942436..."


In [97]:
df2 = json_normalize(df['Response'])
df2.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,status,uploadTime,docId,periodStart,periodEnd,ocrFile,fileName,bankStatement,error,totalPages,message,fileCount,data
0,Rejected,NaN,None,None,None,False,None,False,True,NaN,Unparsable Zip.,0.0,NaN
1,Submitted,1.594244e+12,DOC00123773,None,None,False,bank_statement1_200709025754771155.pdf,False,False,4.0,None,1.0,NaN
2,Processed,NaN,DOC00123773,None,None,NaN,NaN,False,False,NaN,None,NaN,"[{'unmatchedCibil': None, 'panNumber': None, '..."


In [98]:
df3 = pd.concat([df['User ID'],df2['data']],axis=1)

In [99]:
df3 = df3[df3['data'].notnull()]

In [100]:
df3.index = pd.RangeIndex(start = 0,stop=319,step=1)
df3.head(3)

,User ID,data
0,104832,"[{'unmatchedCibil': None, 'panNumber': None, '..."
1,434859,"[{'unmatchedCibil': None, 'panNumber': None, '..."
2,293921,"[{'unmatchedCibil': None, 'panNumber': None, '..."


In [ ]:
bank_names = []
for i in range(df3.shape[0]):
  bank_names.append(df3['data'][i][0]['bankName'])
bank_names

In [ ]:
df3['data'][260][0] #[-1]['totalSalary']

In [103]:
df3['data'][260][0]['salary'] #[-1]['totalSalary']

[]

In [ ]:
salary = []
for i in range(df3.shape[0]):
  if df3['data'][i][0]['salary']:
    salary.append(df3['data'][i][0]['salary'][-1]['totalSalary'])
  else:
    salary.append(-1)
salary

In [ ]:
account_nos = []
for i in range(df3.shape[0]):
  account_nos.append(df3['data'][i][0]['accountNumber'])
account_nos

In [ ]:
account_name = []
for i in range(df3.shape[0]):
  account_name.append(df3['data'][i][0]['accountName'])
account_name

In [107]:
account_name = np.array(account_name)
account_nos = np.array(account_nos)
bank_names = np.array(bank_names)
salary = np.array(salary)

In [108]:
final_df = pd.DataFrame({'User-ID':df3['User ID'],'Bank-Name':bank_names,'Account-Holder-Name':account_name,'Account-No':account_nos,'Recent-Salary':salary})

In [109]:
final_df

,User-ID,Bank-Name,Account-Holder-Name,Account-No,Recent-Salary
0,104832,HDFC,SHANKAR VITTHAL BANSODE,50100287183763,11553.0
1,434859,STANDARD CHARTERED,KUMAR NITESH,27011717278,18271.0
2,293921,HDFC,PAWAN KUMAR RAI,50100330886677,17436.0
3,172850,ICICI,FARIDUDDIN CHAUDHARY,197401503617,-1.0
4,63085,ICICI,RUPESH YENUGU,059601506728,-1.0
...,...,...,...,...,...
314,321522,IDFC,R Prabu,10038821419,14078.0
315,293171,ICICI,None,001401538521,41627.0
316,539352,SBI,MARI KANAKA DURGA,00000062308585354,-1.0
317,103270,HDFC,RUPA YASH SHUKLA,50100257779051,27500.0


In [90]:
final_df.to_csv('v3_final.csv',index=False)